# 平均隐式公式解含时间的薛定谔方程
参考：

计算物理基础(彭芳麟)p336-338

计算物理学p127（Steven E. Koonin，秦克诚译）

一维运动粒子贯穿势垒的薛定谔方程为
$$i\hbar \frac{\partial\varphi}{\partial t} = -\frac{\hbar}{2m}\nabla^2\varphi+V\varphi=H\varphi$$
算符$H=\big(-\frac{\hbar}{2m}\frac{\partial^2}{\partial x^2}+V\big)$
取$\hbar=1, 2m=1$, 化简方程得
$$\frac{\partial\varphi}{\partial t} = -i\big(-\frac{\partial^2}{\partial x^2}+V\big)\varphi=-iH\varphi$$

利用平均公式求解，得到
$$\varphi_{i,j+1}=\bigg(\frac{1-i\frac{1}{2}H\Delta t}{1+i\frac{1}{2}H\Delta t}\bigg)\varphi_{i,j}=\bigg(\frac{2}{1+i\frac{1}{2}H\Delta t}-1\bigg)\varphi_{i,j} = \frac{2\varphi_{i,j}}{1+i\frac{1}{2}H\Delta t}-\varphi_{i,j}$$

令$$\chi=\frac{2\varphi_{i,j}}{1+i\frac{1}{2}H\Delta t}$$
上式可化为
$$\varphi_{i,j+1} = \chi-\varphi_{i,j}$$
且有
$$\big(1+\frac{i}{2}H\Delta t\big)\chi=2\varphi_{i,j}$$

利用差分格式，有
$$H\chi=\big(-\frac{\partial^2}{\partial x^2}+V\big)\chi =\frac{1}{h^2}( \chi_{i+1}-2\chi_{i}+\chi_{i}） + V\chi_{i}$$
其中$h$为$x$方向步长

则$$(1+i\frac{1}{2}H\Delta t)\chi=2\varphi_{i,j}$$
可以显式地写为
$$-\frac{i\Delta t}{2h^2}\chi_{i+1}+\bigg(1+\frac{i\Delta t}{h^2} +\frac{i\Delta t}{2}V\bigg)\chi_{i}-\frac{i\Delta t}{2h^2}\chi_{i-1}=2\varphi_{i,j}$$
上式除以$-i\Delta t/2h^2$, 化为
$$\chi_{i+1}+\bigg(-2+\frac{2ih^2}{\Delta t}-h^2V\bigg)\chi_i+\chi_{i-1} = \frac{4ih^2}{\Delta t}\varphi_{i,j}$$

构造3对角矩阵
$$\begin{pmatrix}
-2+\frac{2ih^2}{\Delta t}-h^2V&1&\ldots&0\\
1&-2+\frac{2ih^2}{\Delta t}-h^2V&\ldots&0\\
\vdots &\vdots & &\vdots\\
0&0&\ldots&-2+\frac{2ih^2}{\Delta t}-h^2V\\
\end{pmatrix}$$

In [1]:
#%%一维势垒贯穿的薛定谔方程
#按照书上p338页的程序写成
import numpy as np, matplotlib.pyplot as plt

def wavefun(x,x0=40,k0=0.5,d=10):
    '''高斯波包函数'''
    return np.exp(k0*1j*x)*np.exp(-(x-x0)**2*np.log10(2)/d**2)

m = 220 #空间坐标的栅格化点数
x = np.arange(m)
v = np.zeros(m)  #势函数
v[m//2:m//2+3] = 1.0   #构建势垒
N = 1040   #时间栅格化点数
#%%
#下面构建矩阵\chi_{j+1}+\bigg(-2+\frac{2ih^2}{\Delta t}\chi_j-h^2v\bigg)+\chi_{j-1}
#取dt, h = 1
A = np.diag(-2+2j-v) + np.diag(np.ones(m-1),1) + np.diag(np.ones(m-1),-1) 

C = np.zeros((x.size, N), complex)   #chi 矩阵
B = np.zeros((x.size, N), complex)   #波函数矩阵

B0 = wavefun(x)  #初始波包
B[:,0] = B0.T

for t in range(N-1):
    C[:,t+1] = 4j*np.linalg.solve(A,B[:,t])
    B[:,t+1] = C[:,t+1] - B[:,t]

#以下动画实现
%matplotlib qt5
from matplotlib import animation
plt.close('all')
fig = plt.figure()
plt.axis([0, m, 0, 1.0])
myline, = plt.plot([],[],'r',lw=2)
myline1, = plt.plot(x, v, 'k', lw=3)
def animate(i):
    myline.set_data(x, np.abs(B[:,i]))
    myline1.set_data(x, v)
    return myline,myline1

anim=animation.FuncAnimation(fig,animate,frames=N, interval=20)

平均隐式公式解热传导方程
$$\left\{
\begin{eqnarray}
\frac{\partial u}{\partial t}  \ \ & = & \ \frac{\partial^2 u}{\partial x^2}, \ \ & 0\leq x \leq 1   \nonumber \\
u(t,0) & = & u(t,1)=0 &  \ \ \ \ \ \ \forall t\nonumber\\
u(0,x) & = & 2x   &  \mathrm{if} \ \ x\leq 0.5 \nonumber\\
u(0,x) & = & 2(1-x)  & \mathrm{if} \ \ x> 0.5   \nonumber
\end{eqnarray}
\right.$$
参考http://www.claudiobellei.com/2016/11/10/crank-nicolson/

已知：
$$u_{i,j+1}=\bigg(\frac{1+\frac{1}{2}L\Delta t}{1-\frac{1}{2}L\Delta t}\bigg)u_{i,j} = \frac{2u_{i,j}}{1-\frac{1}{2}L\Delta t} - u_{i,j}$$
其中$$L = \frac{a^2}{\Delta x^2}(u_{i+1,j}-2u_{i,j}+u_{i-1,j})$$

令$$\chi = \frac{2u_{i,j}}{1-\frac{1}{2}L\Delta t}$$, 方程化为$$u_{i,j+1}=\chi-u_{i,j}$$
现在只需求出$\chi$, 有
$$(1-\frac{1}{2}L\Delta t)\chi=2u_{i,j}$$

即$$\chi_i-\frac{\Delta t}{2}\frac{a^2}{\Delta x^2}(\chi_{i+1}-2\chi_{i}+\chi_{i-1})=2u_{i,j}$$

两边同时除以$-\frac{\Delta t}{2}\frac{a^2}{\Delta x^2}$得到：

$$\chi_{i+1}-(2+\frac{2\Delta x^2}{a^2\Delta t})\chi_i + \chi_{i-1} = -\frac{4\Delta x^2}{a^2\Delta t}u_{i,j}$$

In [1]:
import numpy as np, matplotlib.pyplot as plt

N = 51 #number of grid points
dt = 5.e-4 #time step
L = 1 #size of grid
nsteps = 620 #number of time steps
dx = L/(N-1) #grid spacing
nplot = 20 #number of timesteps before plotting
a2 = 1 #assuming heat diffusion coefficient == 1
r = dx**2/a2/dt 

#initialize grid
x = np.linspace(0,1,N)
#initial condition
u = np.asarray([2*xx if xx<=0.5 else 2*(1-xx) for xx in x])

u[0] = 0
u[-1] = 0
A = np.diag(np.ones(N-2)*(-2-2*r))+np.diag(np.ones(N-3),1) \
   + np.diag(np.ones(N-3),-1) #构造对角矩阵，考虑边界条件，对角矩阵不包括边界条件区域
    
%matplotlib qt5
from matplotlib import animation
plt.close('all')
fig = plt.figure()
plt.axis([0, 1, 0, 1.2])
myline, = plt.plot(x,u,'b',lw=2)

def animate(i):
    c = -4*r*np.linalg.solve(A,u[1:-1])
    u[1:-1] = c - u[1:-1] 
    myline.set_data(x, u)
    return myline

anim=animation.FuncAnimation(fig,animate,frames=nsteps, interval=20) 

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\animation.py", line 1003, in _start
    self.event_source.add_callback(self._step)
AttributeError: 'NoneType' object has no attribute 'add_callback'


## 用傅里叶变换方法解偏微分方程
参考 http://www.thefouriertransform.com/applications/pde.php
参考 https://jakevdp.github.io/blog/2012/09/05/quantum-python/

heat flow on a rod is governed by the partial differential equation
$$\frac{\partial g(x,t)}{\partial t}=c\frac{\partial^2 g(x,t)}{\partial x^2} \tag{1}$$
that is:
$$g_t = cg_{xx}$$
suppose we know the temperature distribution on the rod for all $x$ at $t=0$
$$h(x)=g(x,0)\tag{2}$$
To accomplish this, we will use the Fourier Transform. To start, let's take the Fourier Transform of Equation (1) with respect to x. That is, we will assume x is the variable and hold t constant. First, we will take the Fourier Transform of the right hand side of Equation (1):
$$\begin{align}
F\{cg_{xx}(x,t)\}&=c\int_{-\infty}^{\infty}g_{xx}(x,t)e^{-2\pi ifx}dx\\
&=(2\pi if)^2c\int_{-\infty}^{\infty}g(x,t)e^{-2\pi ifx}dx\\
&=-4c\pi^2f^2G(f,t)\tag{Eq 3}
\end{align}$$

The next step is to take the Fourier Transform (again, with respect to x) of the left hand side of equation (1). 
$$\begin{align}
F\{g_{t}(x,t)\}&=\int_{-\infty}^{\infty}\frac{\partial g(x,t)}{\partial t}e^{-2\pi ifx}dx\\
&=\frac{\partial }{\partial t}\int_{-\infty}^{\infty}g(x,t)e^{-2\pi ifx}dx\\
&=\frac{\partial }{\partial t}G(f,t)\tag{Eq 4}
\end{align}$$

Now that we have the full Fourier Transform of Equation [1], we set Equation [3] equal to Equation [4]:
$$\frac{\partial }{\partial t}G(f,t)=-4c\pi^2f^2G(f,t)\tag{Eq 5}$$

Equation [5] represents a simple Ordinary Differential Equation in the variable t; the function we seek is G(f,t). Using elementary differential equation methods, we obtain:

$$G(f,t)=G(f,0)e^{-4\pi^2f^2ct}\tag{Eq 6}$$

The function G(f, 0) represents the initial condition for the differential Equation in [5]. We also know that G(f, 0) is just the Fourier Transform with respect to x of g(x, 0); and from Equation [2] we know that g(x, 0) is just h(x). Hence, we re-write the initial condition as the Fourier Transform of h(x), and call it H(f):

$$G(f,t)=H(f)e^{-4\pi^2f^2ct}\tag{Eq 7}$$

note that Equation [7] represents the product of two functions: H(f) and the Gaussian Function.
we will take the inverse Fourier Transform of the Gaussian in Equation [7]
$$F^{-1}\big\{ e^{-4\pi^2f^2ct} \big\}=\frac{1}{4\pi ct}e^{-\frac{x^2}{4ct}}\tag{Eq 8}$$

Equation [8] can be used to take the inverse Fourier Transform of Equation [7]:
$$\begin{align}
g(x,t)&=F^{-1}\big\{H(f)e^{-4\pi^2f^2ct}\big\}\\
&=h(x)*\frac{e^{-x^2/(4ct)}}{\sqrt{4\pi ct}}\\
&=\int_{-\infty}^{\infty}h(z)\frac{e^{-(z-x)^2/(4ct)}}{\sqrt{4\pi ct}}dz
\end{align}$$
The result in Equation [9] represents the general solution of Equation [1], subject to the condition of Equation [2]. The result, in essence, represents how the initial heat distribution smooths itself out over time. That is, as t approaches 0, g(x,t) approaches h(x). This is because the Gaussian function becomes very sharp and approximates an impulse function. As time increases, the integration of Equation [9] represents a "smoothing out" or averaging of the initial temperature distribution h(x). This equation dictates precisely how heat spreads out on a rod.

### 用傅里叶变换和逆变换求解热传导方程

In [3]:
import numpy as np, matplotlib.pyplot as plt

N = 51 #number of grid points
dt = 5.e-3 #time step
L = 1 #size of grid
nsteps = 620 #number of time steps
dx = L/(N-1) #grid spacing
a2 = 1 #assuming heat diffusion coefficient == 1
r = dx**2/a2/dt 

x = np.linspace(-.5,.5,N)
t = np.arange(dt,nsteps*dt+dt,dt)
u0 = np.array([2*xx if xx<=0.0 else -2*xx for xx in x])+1 #初始条件

#plt.plot(x,u0)

from scipy.fftpack import fft,ifft2

#求频率的数组
#$k_0 = - \pi / \Delta x$
#$k_m = k_0 + m\Delta k$, with $\Delta k = 2\pi / (N\Delta x)$
k0 = -np.pi/dx
dk = 2*np.pi/(N*dx)
k = [k0]
for i in range(1,N):
    k.append(k0+i*dk)
k = np.array(k) #频率数组
K, T = np.meshgrid(k, t)
phi1 =np.exp(-4*np.pi*np.pi*K*K*T) #频域的波函数

Hf = fft(u0)  #初始条件的傅里叶变换
g_xt = ifft2(Hf*phi1)  #傅里叶逆变换求波函数
gt = np.abs(g_xt)
%matplotlib qt5
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib import cm

fig = plt.figure(figsize=(8,6))

ax1 = fig.add_subplot(111, projection='3d')
X,T = np.meshgrid(x,t)
surf = ax1.plot_surface(X, T, gt, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

ax1.set_xlabel('x')
ax1.set_ylabel('t')
ax1.set_zlabel('u')

Text(0.5, 0, 'u')

下面是用卷积的部分解PDE, 总结：不知道为什么结果很不理想，也许是convolve函数的原因

In [5]:
import numpy as np, matplotlib.pyplot as plt

N = 51 #number of grid points
dt = 5.e-3 #time step
L = 1 #size of grid
nsteps = 620 #number of time steps
dx = L/(N-1) #grid spacing
a2 = 1 #assuming heat diffusion coefficient == 1
r = dx**2/a2/dt 

x = np.linspace(-.5,.5,N)
t = np.arange(dt,nsteps*dt+dt,dt)
u = np.ones((nsteps,N))
u[0,:] = np.asarray([2*xx if xx<=0.0 else -2*xx for xx in x])+1
for i in range(1,u.shape[0]):
    u[i,:] = u[0,:]

#plt.plot(x,u[0,:])
plt.imshow(u)
#%%
from scipy.signal import fftconvolve

x,t = np.meshgrid(x,t)
gt=fftconvolve(u, np.exp(-x**2/4/t)/np.sqrt(4*np.pi*t), mode='same')

#gt=np.exp(-x**2/4/t)/np.sqrt(4*np.pi*t)

plt.imshow(gt)
#%%
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib import cm

fig = plt.figure(figsize=(8,6))

ax1 = fig.add_subplot(111, projection='3d')

surf = ax1.plot_surface(x, t, gt, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

ax1.set_xlabel('x')
ax1.set_ylabel('t')
ax1.set_zlabel('u')

Text(0.5,0,'u')

总结：不知道为什么结果很不理想，也许时convolve函数的原因